# Chat with your code! </>


<img src="chat_with_code.png" width=800px>

In [1]:
import os

os.environ["HF_HOME"] = "/teamspace/studios/this_studio/weights"
os.environ["TORCH_HOME"] = "/teamspace/studios/this_studio/weights"

import gc
import re
import uuid
import textwrap
import subprocess
import nest_asyncio
from dotenv import load_dotenv
from IPython.display import Markdown, display

from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.core import PromptTemplate
from llama_index.core import SimpleDirectoryReader
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core import VectorStoreIndex
from llama_index.core.storage.storage_context import StorageContext

from langchain.embeddings import HuggingFaceEmbeddings
from llama_index.embeddings.langchain import LangchainEmbedding

from rag_101.retriever import (
    load_embedding_model,
    load_reranker_model
)

In [2]:
# allows nested access to the event loop
nest_asyncio.apply()

In [3]:
# setting up the llm
llm=Ollama(model="mistral", request_timeout=60.0)

# setting up the embedding model
lc_embedding_model = load_embedding_model()
embed_model = LangchainEmbedding(lc_embedding_model)

In [4]:
# utility functions
def parse_github_url(url):
    pattern = r"https://github\.com/([^/]+)/([^/]+)"
    match = re.match(pattern, url)
    return match.groups() if match else (None, None)

def clone_github_repo(repo_url):    
    try:
        print('Cloning the repo ...')
        result = subprocess.run(["git", "clone", repo_url], check=True, text=True, capture_output=True)
    except subprocess.CalledProcessError as e:
        print(f"Failed to clone repository: {e}")
        return None


def validate_owner_repo(owner, repo):
    return bool(owner) and bool(repo)

In [5]:
# Setup a query engine

def setup_query_engine(github_url):
    
    owner, repo = parse_github_url(github_url)
    
    if validate_owner_repo(owner, repo):
        # Clone the GitHub repo & save it in a directory
        input_dir_path = f"/teamspace/studios/this_studio/{repo}"

        if os.path.exists(input_dir_path):
            pass
        else:
            clone_github_repo(github_url)
        
        loader = SimpleDirectoryReader(
            input_dir = input_dir_path,
            required_exts=[".py", ".ipynb", ".js", ".ts", ".md"],
            recursive=True
        )

        try:
            docs = loader.load_data()

            # ====== Create vector store and upload data ======
            Settings.embed_model = embed_model
            index = VectorStoreIndex.from_documents(docs, show_progress=True)
            # TODO try async index creation for faster emebdding generation & persist it to memory!
            # index = VectorStoreIndex(docs, use_async=True)

            # ====== Setup a query engine ======
            Settings.llm = llm
            query_engine = index.as_query_engine(similarity_top_k=4)
            
            # ====== Customise prompt template ======
            qa_prompt_tmpl_str = (
            "Context information is below.\n"
            "---------------------\n"
            "{context_str}\n"
            "---------------------\n"
            "Given the context information above I want you to think step by step to answer the query in a crisp manner, incase case you don't know the answer say 'I don't know!'.\n"
            "Query: {query_str}\n"
            "Answer: "
            )
            qa_prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str)

            query_engine.update_prompts(
                {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
            )

            if docs:
                print("Data loaded successfully!!")
                print("Ready to chat!!")
            else:
                print("No data found, check if the repository is not empty!")
            
            return query_engine

        except Exception as e:
            print(f"An error occurred: {e}")
    else:
        print('Invalid github repo, try again!')
        return None

In [6]:
# Provide url to the repository you want to chat with
github_url = "https://github.com/Lightning-AI/lit-gpt"

query_engine = setup_query_engine(github_url=github_url)

Cloning the repo ...


Parsing nodes:   0%|          | 0/229 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/387 [00:00<?, ?it/s]

Data loaded successfully!!
Ready to chat!!


In [7]:
response = query_engine.query('Can you provide a step by step guide to finetuning an llm using lit-gpt')
display(Markdown(str(response)))

 To finetune a Large Language Model (LLM) using Lit-GPT, follow these steps:

1. Install the required packages: First, install EleutherAI's lm-eval framework by running the following command in your terminal or command prompt:
   ```bash
   pip install https://github.com/EleutherAI/lm-evaluation-harness/archive/refs/heads/master.zip -U
   ```
2. Familiarize yourself with the conceptual tutorials: To gain a better understanding of finetuning, read the articles available in the `README.md` file under the `tutorials` directory. These include "Understanding Parameter-Efficient Finetuning of Large Language Models" and "Parameter-Efficient LLM Finetuning With Low-Rank Adaptation (LoRA)".
3. Choose a finetuning method: Lit-GPT supports various finetuning methods, including using adapters and LoRA or QLoRA. Familiarize yourself with these methods by reading the corresponding how-to guides available in the `README.md` file under the `tutorials/finetune` directory.
4. Prepare your dataset: Make sure you have a suitable dataset for finetuning the LLM. The data should be formatted correctly and easily loadable by Lit-GPT.
5. Finetune with Adapters (Optional): If you choose to use adapters for finetuning, follow the instructions provided in the "Finetune with Adapters" guide in the `README.md` file under the `tutorials/finetune` directory.
6. Finetune with LoRA or QLoRA (Optional): If you choose to use LoRA or QLoRA for finetuning, follow the instructions provided in the "Finetune with LoRA or QLoRA" guide in the `README.md` file under the `tutorials/finetune` directory.
7. Evaluate your finetuned model: Once you have finished finetuning your LLM, evaluate its performance using EleutherAI's lm-eval framework. Use the following command to evaluate the model on all tasks in Eleuther AI's Evaluation Harness:
   ```bash
   python eval/lm_eval_harness.py \
       --checkpoint_dir "[path_to_your_finetuned_model_directory]" \
       --precision "bf16-true" \
       --save_filepath "results.json"
   ```
   To evaluate the model on specific tasks, use the `--eval_tasks` flag:
   ```bash
   python eval/lm_eval_harness.py \
       --checkpoint_dir "[path_to_your_finetuned_model_directory]" \
       --eval_tasks "[task1, task2]" \
       --precision "bf16-true" \
       --save_filepath "results.json"
   ```
Replace `[path_to_your_finetuned_model_directory]` with the path to your finetuned model directory, and replace `task1` and `task2` with the names of the evaluation tasks you want to use.
8. (Optional) Analyze the results: Once you have evaluated your model, analyze the results provided in the output file to assess its performance on various tasks.